In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# To prevent the annoying warning from scikit learn package
import warnings 
warnings.filterwarnings('ignore')

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('darkgrid')
cmap = sns.cm.mako_r

%matplotlib inline

from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

**2. Importing the Data using Pandas read_csv(). And calling head() and info() on the DataFrame**

In [ ]:
df= pd.read_csv("/kaggle/input/hr-analytics-job-change-of-data-scientists/aug_train.csv")

In [ ]:
df.head()

In [ ]:
df.info()

As we can see that there are a lot of missing values and most of them are Categorical and it is not feasible to fill those values as we may end up providing completly wrong data to the training set. So, I prefer dropping them instead.

In [ ]:
df.isnull().sum()

In [ ]:
#will check percentage of null value
(df.isnull().sum()/len(df))*100

In [ ]:
df.shape

In [ ]:
df.size

In [ ]:
###for i in df.columns:
   ## print(df.nunique())
    

In [ ]:
for col in df.columns:
    print(col, df[col].unique())

# data visulization

### Which gender is more likely to move for a new job?

In [ ]:
sns.countplot(df.gender)

In [ ]:
genders = df[df['target'] == 1]['gender']
temp_df= (genders.value_counts())/len(genders)*100

In [ ]:
temp_df

In [ ]:
plt.figure(figsize=(8,8))
plt.pie(temp_df,labels = temp_df.keys() , colors = ['blue','yellow','red'], autopct="%.2f%%",pctdistance=0.8)
plt.title('Gender % looking for new job', fontsize=20)

In [ ]:
male_newjob = df[(df['gender']=='Male') & df['target']==1]
female_newjob = df[(df['gender']=='Female') & df['target']==1]

# print
print('{} % of male who are looking for a new job'.format(len(male_newjob)/len(df['gender']=='Male')*100))
print('{} % of female who are looking for a new job'.format(len(female_newjob)/len(df['gender']=='Female')*100))

## From which company type people are looking for new job?

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(df['company_type'])
plt.show()

here we can see that most of pvt sector employees are looking for job change

In [ ]:
company_type = df[df['target'] == 1]['company_type']
temp = company_type.value_counts()
labels = temp.keys()
bar,ax = plt.subplots(figsize=(8,8))
plt.pie(x = temp, labels = labels, autopct="%.1f%%",pctdistance=0.7)
plt.title('People leaving company', fontsize=20)

In [ ]:
for i in df['company_type'].unique():
    company_newjob = df[(df['company_type']==i) & df['target']==1]
    print('{} % of {} who are looking for a new job'.format(len(company_newjob)/len(df['company_type']==i)*100,i))

In [ ]:
sns.countplot(df['relevent_experience'])

In [ ]:
(df['relevent_experience']).value_counts()

In [ ]:
sns.countplot(df['relevent_experience'],hue=df['target'])
plt.xlabel('target')
plt.ylabel('count')
plt.title('Relevent experience on the basis of target')

In [ ]:
yes_newjob = df[(df['relevent_experience']=='Has relevent experience') & df['target']==1]
no_newjob = df[(df['relevent_experience']=='No relevent experience') & df['target']==1]

# print
print('{} % of having relevant experience who are looking for a new job'.format(len(yes_newjob)/len(df['relevent_experience']=='Has relevent experience')*100))
print('{} % of not havinf relevant experience who are looking for a new job'.format(len(no_newjob)/len(df['relevent_experience']=='No relevent experience')*100))

### Did any people got into data science field without having graduation degree?

In [ ]:
sns.countplot(df['education_level'])

In [ ]:
df.education_level.value_counts()

In [ ]:
people_withoutdegree = df[(df['education_level'] == 'Primary School')& (df['education_level']=='High School') & (df['enrolled_university'] == "no_enrollment")]
print("People who have got into the data science world without graduation are", len(people_withoutdegree))

So their is not a single person who get into this field without graduation.

### Years between last and current job?

In [ ]:
sns.countplot(df['last_new_job'])

# DATA PREPROCESSING

First of all we are going to drop unnecessary columns,so we don't require enrollee_id and city column

In [ ]:
df.drop(columns=["city","enrollee_id"],inplace=True)

### Countplot for some categorical feature

We already have seen countplot for various features.Now,we are going to see countplot for the features we haven't seen yet

In [ ]:
sns.countplot(df['enrolled_university'])


So it's seems like most people wjo are currently doing job haven't enrolled in any university

In [ ]:

sns.countplot(df['major_discipline'])

Most of the Candidates are from STEM. That is their major discipline was in one of the Following:<br>
<ol>
    <li><span style='color:red'>S</span>cience</li>
    <li><span style='color:red'>T</span>echnology</li>
    <li><span style='color:red'>E</span>ngineering</li>
    <li><span style='color:red'>M</span>athematics</li>
</ol>

In [ ]:
sns.countplot(df['experience'])

### dealing with null values

In [ ]:
# null value
percent_null = df.isnull().mean()*100
print(percent_null)

Columns in which we have 2% or less than 2% null values we can drop those null values

In [ ]:
df.dropna(subset=['enrolled_university','education_level','experience','last_new_job'], axis=0, inplace=True)

In [ ]:
# after dropping those null values
df.shape

Now we are going to fill null values with their mode as all the columns left have dtype as 'object'

In [ ]:
df.dtypes

In [ ]:
col_mode = ['gender','company_size','major_discipline','company_type','relevent_experience']
for col in col_mode:
    df[col].fillna(df[col].mode()[0],inplace=True)


Let's change the dtype of experience and last_new_job column

In [ ]:
df.replace(to_replace = 'Has relevent experience',value = 'Yes',inplace = True)
df.replace(to_replace = 'No relevent experience',value='No',inplace = True )

df.replace(to_replace = '<1',value = '0',inplace = True)
df.replace(to_replace = '>20',value = '21',inplace=True)
df.replace(to_replace = 'never',value = '0',inplace=True)
df.replace(to_replace = '>4',value = '5',inplace=True)

df.replace(to_replace = '<10',value = 'around_10',inplace=True)
df.replace(to_replace = '10/49',value = 'around_50',inplace=True)
df.replace(to_replace = '50-99',value = 'around_100',inplace=True)
df.replace(to_replace = '100-500',value = 'around_500',inplace=True)
df.replace(to_replace = '500-999',value = 'around_1000',inplace=True)
df.replace(to_replace = '1000-4999',value = 'around_5000',inplace=True)
df.replace(to_replace = '5000-9999',value = 'around_10000',inplace=True)
df.replace(to_replace = '10000+',value = 'more_than_10000',inplace=True)

df.replace(to_replace = 'Full time course',value = 'Full_time_course',inplace=True)
df.replace(to_replace = 'Part time course',value = 'Part_time_course',inplace=True)

df.replace(to_replace = 'Primary School',value = 'Primary_School',inplace=True)
df.replace(to_replace = 'High School',value = 'High_School',inplace=True)

df.replace(to_replace = 'Business Degree',value = 'Business_Degree',inplace=True)
df.replace(to_replace = 'No Major',value = 'No_Major',inplace=True)

df.replace(to_replace = 'Pvt Ltd',value = 'Pvt_Ltd',inplace=True)
df.replace(to_replace = 'Funded Startup',value = 'Funded_Startup',inplace=True)
df.replace(to_replace = 'Public Sector',value = 'Public_Sector',inplace=True)
df.replace(to_replace = 'Early Stage Startup',value = 'Early_Stage_Startup',inplace=True)

df['major_discipline'].replace('Other','Other_major',inplace=True)
df['company_type'].replace('Other','Other_type',inplace=True)

In [ ]:
df = df.astype({'experience':int,'last_new_job':int})

### Handling Categorical Values

In [ ]:
# get dummies

education_df = pd.get_dummies(df[['education_level']],drop_first=True,prefix=[None])
company_size_df = pd.get_dummies(df[['company_size']],drop_first=True,prefix=[None])
company_type_df = pd.get_dummies(df[['company_type']],drop_first=True,prefix=[None])
major_df = pd.get_dummies(df[['major_discipline']],drop_first=True,prefix=[None])
university_df = pd.get_dummies(df[['enrolled_university']],drop_first=True,prefix=[None])
experience_df = pd.get_dummies(df[['relevent_experience']],drop_first=True,prefix=[None])
gender_df = pd.get_dummies(df[['gender']],drop_first=True,prefix=[None])

In [ ]:
# drop original columns
df.drop(['education_level','company_size','company_type','major_discipline','enrolled_university','relevent_experience','gender'],axis=1,inplace=True)

In [ ]:
final_df = pd.concat([df,education_df,company_size_df,company_type_df,major_df,university_df,experience_df,gender_df],axis=1) 

In [ ]:
final_df.head(5)

In [ ]:
final_df.to_csv('final_df.csv')

In [ ]:
X = final_df.drop(['target'], axis = 1)
Y = final_df['target']

### corelation

In [ ]:
X.corr()

In [ ]:
plt.figure(figsize=(16,8))
sns.heatmap(X.corr())

# check multicolineartiy

In [ ]:
from statsmodels.stats.outliers_influence import variance_inflation_factor    

def calculate_vif_(X, thresh=5.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

In [ ]:
calculate_vif_(X)

lets check data is balanced or not

In [ ]:
sns.countplot(df['target'])

data is imblanced.

In [ ]:
from imblearn.over_sampling import SMOTE
smote = SMOTE(random_state = 402)
X_smote, Y_smote = smote.fit_resample(X,Y)


sns.countplot(Y_smote)

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X_smote, Y_smote, test_size = 0.2 ,random_state = 42)

# standard scaler

In [ ]:
scaler=StandardScaler()
scaler.fit(X_train)
#scaler.fit_transform(X_train)
X_train = scaler.transform(X_train)
X_test=scaler.transform(X_test)

# KNN CLASSIFIER

In [ ]:
#predicting using the KNeighbors_Classifier
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
model_KNN=KNeighborsClassifier(n_neighbors=int(np.sqrt(len(X_train))), 
                               metric='minkowski')
#euclidean,manhattan,minkowski
#fit the model on the data and predict the values
model_KNN.fit(X_train,Y_train)

Y_pred=model_KNN.predict(X_test)
print(list(zip(Y_test,Y_pred)))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

cfm=confusion_matrix(Y_test,Y_pred)
print(cfm)
print()

print("Classification report: ")

print(classification_report(Y_test,Y_pred))

acc=accuracy_score(Y_test, Y_pred)
print("Accuracy of the model: ",acc)

 we will try to improve accuracy 

In [ ]:
from sklearn.metrics import accuracy_score
my_dict={}
for K in range(1,30):
    model_KNN = KNeighborsClassifier(n_neighbors=K,metric="euclidean")
    model_KNN.fit(X_train, Y_train) 
    Y_pred = model_KNN.predict(X_test)
    print ("Accuracy is ", accuracy_score(Y_test,Y_pred), "for K-Value:",K)
    my_dict[K]=accuracy_score(Y_test,Y_pred)

In [ ]:
my_dict

In [ ]:
for k in my_dict:
    if my_dict[k]==max(my_dict.values()):
        print("KNN CLASSIFIER MAX ACCURACY IS : ",k,":",my_dict[k])

# LOGISTIC REGRESSION

In [ ]:
from sklearn.linear_model import LogisticRegression
#create a model
classifier=LogisticRegression()
#fitting training data to the model
classifier.fit(X_train,Y_train)

Y_pred=classifier.predict(X_test)
print(list(zip(Y_test,Y_pred)))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

confusion_matrix=confusion_matrix(Y_test,Y_pred)
print(confusion_matrix)
print()
print("Classification report: ")

print(classification_report(Y_test,Y_pred))

accuracy_score=accuracy_score(Y_test, Y_pred)
print("Accuracy of the model: ",accuracy_score)

In [ ]:
import seaborn as sns
sns.heatmap(confusion_matrix, annot=True)

# SVM

In [ ]:
from sklearn.svm import SVC
svc_model=SVC(kernel='linear',C=10.0,gamma=0.001)
svc_model.fit(X_train, Y_train)
Y_pred=svc_model.predict(X_test)
print(list(Y_pred))

In [ ]:
svc_model.score(X_train,Y_train)


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report

confusion_matrix=confusion_matrix(Y_test,Y_pred)
print(confusion_matrix)
print()
print("Classification report: ")

print(classification_report(Y_test,Y_pred))

accuracy_score=accuracy_score(Y_test, Y_pred)
print("Accuracy of the model: ",accuracy_score)

# Decision Trees.

In [ ]:
#predicting using the Decision_Tree_Classifier
from sklearn.tree import DecisionTreeClassifier

model_DecisionTree=DecisionTreeClassifier(criterion="gini",random_state=10,splitter="best")

#fit the model on the data and predict the values
model_DecisionTree.fit(X_train,Y_train)
Y_pred=model_DecisionTree.predict(X_test)
print(Y_pred)
#print(list(zip(Y_test,Y_pred)))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
#confusion matrix
print(confusion_matrix(Y_test,Y_pred))
print(accuracy_score(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))

In [ ]:
model_DecisionTree.score(X_train,Y_train)

# Random_Forest_Classifier

In [ ]:
#predicting using the Random_Forest_Classifier
from sklearn.ensemble import RandomForestClassifier

model_RandomForest=RandomForestClassifier(n_estimators=50, random_state=10)

#fit the model on the data and predict the values
model_RandomForest.fit(X_train,Y_train)

Y_pred=model_RandomForest.predict(X_test)


In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
#confusion matrix
print(confusion_matrix(Y_test,Y_pred))
print(accuracy_score(Y_test,Y_pred))
print(classification_report(Y_test,Y_pred))

Random forrest accuracy is 82.83

# gredieant boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
clf = GradientBoostingClassifier(n_estimators=100, learning_rate=1.0,
...     max_depth=1, random_state=0).fit(X_train, Y_train)
y_pred_grd_test=clf.predict(X_test)
>>> 

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score,classification_report
#confusion matrix
print(confusion_matrix(Y_test,y_pred_grd_test))
print(accuracy_score(Y_test,y_pred_grd_test))
print(classification_report(Y_test,y_pred_grd_test))

#  THANK YOU